In [1]:
import os
from dotenv import load_dotenv, find_dotenv

import pandas as pd
from httpx import Client, HTTPTransport, Proxy, Timeout, ProxyError

load_dotenv(find_dotenv())

True

In [2]:
def create_client() -> Client:
    """
    Creates a common client for future http requests

    Returns:
        Client: client with ford proxies
    """
    ford_proxy = str(os.getenv("FORD_PROXY"))
    timeout_config = Timeout(30.0)
    proxy_mounts = {
        "http://": HTTPTransport(proxy=Proxy(ford_proxy)),
        "https://": HTTPTransport(proxy=Proxy(ford_proxy)),
    }
    return Client(
        timeout=timeout_config,
        mounts=proxy_mounts,
        verify=False,
    )


def get_token():
    with create_client() as client:
        response = client.post(
            str(os.getenv("TOKEN_ENDPOINT")),
            data={
                "client_id": str(os.getenv("CLIENT_ID")),
                "client_secret": str(os.getenv("CLIENT_SECRET")),
                "scope": str(os.getenv("SCOPE")),
                "grant_type": "client_credentials",
            },
            timeout=160,
        )
    print(
        "token expires in:",
        round(int(response.json()["expires_in"]) / 60, 0),
        "minutes",
    )
    return response.json()["access_token"]

In [3]:
def call_api_movement(complaint: str, client: Client, token: str):
    content = {
        "model": "gpt-4",
        "context": (
            "You are a helpful text reader and analyzer. Resume this complaint"
        ),  # sets the overall behavior of the assistant.
        "messages": [
            {"role": "user", "content": f"resume the following complaint: {complaint}"}
        ],
        "parameters": {
            "temperature": 0.05,  # Determines the randomnes of the model's response.
        },
    }
    try:
        response = client.post(
            str(os.getenv("API_ENDPOINT")),
            headers={"Authorization": f"Bearer {token}"},
            json=content,
            timeout=20,
        )
        message = response.json()["content"]
        print(message)
        if "\n" in message:
            message = message.split("\n")[0]

        return message

    except ProxyError as exc:
        print(exc)

In [4]:
df = pd.read_csv("../../../../data/raw/mock_dataset.csv")
token = get_token()
with create_client() as client:
    df["Resume"] = df["CDESCR"].apply(lambda row: call_api_movement(row, client, token))

df.head()

503 Service Unavailable
503 Service Unavailable
503 Service Unavailable
503 Service Unavailable
503 Service Unavailable
503 Service Unavailable
503 Service Unavailable


,CMPLID,ODINO,MFR_NAME,MAKETXT,MODELTXT,YEARTXT,CRASH,FAILDATE,FIRE,INJURED,...,DEALER_NAME,DEALER_TEL,DEALER_CITY,DEALER_STATE,DEALER_ZIP,PROD_TYPE,REPAIRED_YN,MEDICAL_ATTN,VEHICLES_TOWED_YN,Resume
0,1971468,11574310,Ford Motor Company,FORD,ESCAPE,2017.0,N,20240301,N,0,...,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N,None
1,1971476,11574744,Ford Motor Company,FORD,F-150,2006.0,N,20240223,N,0,...,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N,None
2,1971482,11573968,Ford Motor Company,FORD,F-150,2018.0,N,20240202,N,0,...,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N,None
3,1971484,11574317,Ford Motor Company,FORD,EDGE,2019.0,N,20231014,N,0,...,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N,None
4,1971491,11574914,Ford Motor Company,FORD,EXPLORER,2018.0,N,20231220,N,0,...,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N,None
